<a href="https://colab.research.google.com/github/BooEiEi/Predicting-Technology-Sector-Stock-Prices/blob/main/Hyperparameter_LightGBM_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DATA

In [1]:
import os# os ใช้จัดการกับไฟล์ ว่าอยู่ที่ไหน
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path_DF = '/content/drive/My Drive/master project/data/DF4'

Mounted at /content/drive


In [11]:
import lightgbm
import sklearn
from lightgbm import LGBMRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error

In [3]:
df_META = pd.read_csv(os.path.join(path_DF, 'df_META.csv'))
df_AAPL= pd.read_csv(os.path.join(path_DF, 'df_AAPL.csv'))
df_MSFT= pd.read_csv(os.path.join(path_DF, 'df_MSFT.csv'))
df_NVDA= pd.read_csv(os.path.join(path_DF, 'df_NVDA.csv'))
df_GOOG= pd.read_csv(os.path.join(path_DF, 'df_GOOG.csv'))

In [4]:
df_META=df_META.drop(['Unnamed: 0'], axis=1)
df_META['date'] = pd.to_datetime(df_META['date'])
df_META = df_META.sort_values('date').reset_index(drop=True)

In [5]:
df_AAPL=df_AAPL.drop(['Unnamed: 0'], axis=1)
df_AAPL['date'] = pd.to_datetime(df_AAPL['date'])
df_AAPL = df_AAPL.sort_values('date').reset_index(drop=True)

In [6]:
df_MSFT=df_MSFT.drop(['Unnamed: 0'], axis=1)
df_MSFT['date'] = pd.to_datetime(df_MSFT['date'])
df_MSFT = df_MSFT.sort_values('date').reset_index(drop=True)

In [7]:
df_NVDA=df_NVDA.drop(['Unnamed: 0'], axis=1)
df_NVDA['date'] = pd.to_datetime(df_NVDA['date'])
df_NVDA = df_NVDA.sort_values('date').reset_index(drop=True)

In [8]:
df_GOOG=df_GOOG.drop(['Unnamed: 0'], axis=1)
df_GOOG['date'] = pd.to_datetime(df_GOOG['date'])
df_GOOG = df_GOOG.sort_values('date').reset_index(drop=True)

#Normalized

In [9]:
train_size = int(len(df_META) * 0.75)
test_size = len(df_META) - train_size

In [12]:
scaler = MinMaxScaler()

In [13]:
X_META = df_META.drop(columns=["Stockprice_META","date"])
y_META = df_META["Stockprice_META"]
X_META['has_news'] = X_META['sentiment_META'].apply(lambda x: 1 if x != 0.0 else 0)

In [14]:
X_AAPL = df_AAPL.drop(columns=["Stockprice_AAPL","date"])
y_AAPL = df_AAPL["Stockprice_AAPL"]
X_AAPL['has_news'] = X_AAPL['sentiment_AAPL'].apply(lambda x: 1 if x != 0.0 else 0)

In [15]:
X_GOOG = df_GOOG.drop(columns=["Stockprice_GOOGL","date"])
y_GOOG = df_GOOG["Stockprice_GOOGL"]
X_GOOG['has_news'] = X_GOOG['sentiment_GOOGL'].apply(lambda x: 1 if x != 0.0 else 0)

In [16]:
X_NVDA = df_NVDA.drop(columns=["Stockprice_NVDA","date"])
y_NVDA = df_NVDA["Stockprice_NVDA"]
X_NVDA['has_news'] = X_NVDA['sentiment_NVDA'].apply(lambda x: 1 if x != 0.0 else 0)

In [17]:
X_MSFT = df_MSFT.drop(columns=["Stockprice_MSFT","date"])
y_MSFT = df_MSFT["Stockprice_MSFT"]
X_MSFT['has_news'] = X_MSFT['sentiment_MSFT'].apply(lambda x: 1 if x != 0.0 else 0)

In [18]:
Xtrain_META, XTest_META = X_META.iloc[:train_size], X_META.iloc[train_size:] # ข้อมูล train
ytrain_META, yTest_META = y_META.iloc[:train_size], y_META.iloc[train_size:] # ข้อมูล test

In [19]:
Xtrain_AAPL, XTest_AAPL = X_AAPL.iloc[:train_size], X_AAPL.iloc[train_size:] # ข้อมูล train
ytrain_AAPL, yTest_AAPL = y_AAPL.iloc[:train_size], y_AAPL.iloc[train_size:] # ข้อมูล test

In [20]:
Xtrain_GOOG, XTest_GOOG = X_GOOG.iloc[:train_size], X_GOOG.iloc[train_size:] # ข้อมูล train
ytrain_GOOG, yTest_GOOG = y_GOOG.iloc[:train_size], y_GOOG.iloc[train_size:] # ข้อมูล test

In [21]:
Xtrain_MSFT, XTest_MSFT = X_MSFT.iloc[:train_size], X_MSFT.iloc[train_size:] # ข้อมูล train
ytrain_MSFT, yTest_MSFT = y_MSFT.iloc[:train_size], y_MSFT.iloc[train_size:] # ข้อมูล test

In [22]:
Xtrain_NVDA, XTest_NVDA = X_NVDA.iloc[:train_size], X_NVDA.iloc[train_size:] # ข้อมูล train
ytrain_NVDA, yTest_NVDA = y_NVDA.iloc[:train_size], y_NVDA.iloc[train_size:] # ข้อมูล test

In [23]:
Xtrain_META = scaler.fit_transform(Xtrain_META)
XTest_META =  scaler.transform(XTest_META)

In [24]:
Xtrain_AAPL = scaler.fit_transform(Xtrain_AAPL)
XTest_AAPL = scaler.transform(XTest_AAPL)

In [25]:
Xtrain_GOOG = scaler.fit_transform(Xtrain_GOOG)
XTest_GOOG = scaler.transform(XTest_GOOG)

In [26]:
Xtrain_MSFT = scaler.fit_transform(Xtrain_MSFT)
XTest_MSFT = scaler.transform(XTest_MSFT)

In [27]:
Xtrain_NVDA = scaler.fit_transform(Xtrain_NVDA)
XTest_NVDA = scaler.transform(XTest_NVDA)

#GOOGL

In [29]:
model = LGBMRegressor(random_state=42, verbosity=-1)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_data_in_leaf': [25, 50, 100, 150, 200],
    'alpha': [0.1, 1, 5, 10, 15],
    'reg_lambda' : [0.1, 1, 5, 10, 15],
    'n_estimators' : [100, 200, 400, 600],
    'feature_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],
    'max_bin' : [50, 100, 150, 200, 400]
}

tscv = TimeSeriesSplit(n_splits=5)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv,
                           scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(Xtrain_GOOG , ytrain_GOOG)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(Xtrain_GOOG)
mse = mean_squared_error(ytrain_GOOG, y_pred)
rmse = root_mean_squared_error(ytrain_GOOG, y_pred)
mae = mean_absolute_error(ytrain_GOOG, y_pred)
mape = mean_absolute_percentage_error(ytrain_GOOG, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Best Hyperparameters: {'alpha': 0.1, 'feature_fraction': 0.6, 'learning_rate': 0.2, 'max_bin': 150, 'max_depth': 3, 'min_data_in_leaf': 25, 'n_estimators': 100, 'reg_lambda': 5}
Test Set Mean Squared Error: 3.3772
Test Set Root Mean Squared Error: 1.8377
Test Set Mean Absolute Error: 1.4083
Test Set Mean Absolute Percentage Error: 0.0116


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


#AAPL

In [30]:
model = LGBMRegressor(random_state=42, verbosity=-1)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_data_in_leaf': [25, 50, 100, 150, 200],
    'n_estimators' : [100, 200, 400, 600],
    'feature_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],
    'max_bin' : [50, 100, 150, 200, 400]
}

tscv = TimeSeriesSplit(n_splits=5)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv,
                           scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(Xtrain_AAPL , ytrain_AAPL)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(Xtrain_AAPL)
mse = mean_squared_error(ytrain_AAPL, y_pred)
rmse = root_mean_squared_error(ytrain_AAPL, y_pred)
mae = mean_absolute_error(ytrain_AAPL, y_pred)
mape = mean_absolute_percentage_error(ytrain_AAPL, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Best Hyperparameters: {'feature_fraction': 0.7, 'learning_rate': 0.2, 'max_bin': 200, 'max_depth': 3, 'min_data_in_leaf': 25, 'n_estimators': 600}
Test Set Mean Squared Error: 0.3977
Test Set Root Mean Squared Error: 0.6306
Test Set Mean Absolute Error: 0.4700
Test Set Mean Absolute Percentage Error: 0.0029


#MSFT

In [31]:
model = LGBMRegressor(random_state=42, verbosity=-1)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_data_in_leaf': [25, 50, 100, 150, 200],
    'n_estimators' : [100, 200, 400, 600],
    'feature_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],
    'max_bin' : [50, 100, 150, 200, 400]
}
tscv = TimeSeriesSplit(n_splits=5)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv,
                           scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(Xtrain_MSFT , ytrain_MSFT)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(Xtrain_MSFT)
mse = mean_squared_error(ytrain_MSFT, y_pred)
rmse = root_mean_squared_error(ytrain_MSFT, y_pred)
mae = mean_absolute_error(ytrain_MSFT, y_pred)
mape = mean_absolute_percentage_error(ytrain_MSFT, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

Best Hyperparameters: {'feature_fraction': 0.6, 'learning_rate': 0.2, 'max_bin': 100, 'max_depth': 3, 'min_data_in_leaf': 25, 'n_estimators': 600}
Test Set Mean Squared Error: 1.6576
Test Set Root Mean Squared Error: 1.2875
Test Set Mean Absolute Error: 1.0090
Test Set Mean Absolute Percentage Error: 0.0033


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


#NVDA

In [32]:
model = LGBMRegressor(random_state=42, verbosity=-1)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_data_in_leaf': [25, 50, 100, 150, 200],
    'n_estimators' : [100, 200, 400, 600],
    'feature_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],
    'max_bin' : [50, 100, 150, 200, 400]
}
tscv = TimeSeriesSplit(n_splits=5)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv,
                           scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(Xtrain_NVDA , ytrain_NVDA)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(Xtrain_NVDA)
mse = mean_squared_error(ytrain_NVDA, y_pred)
rmse = root_mean_squared_error(ytrain_NVDA, y_pred)
mae = mean_absolute_error(ytrain_NVDA, y_pred)
mape = mean_absolute_percentage_error(ytrain_NVDA, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

Best Hyperparameters: {'feature_fraction': 0.5, 'learning_rate': 0.2, 'max_bin': 100, 'max_depth': 3, 'min_data_in_leaf': 25, 'n_estimators': 600}
Test Set Mean Squared Error: 0.1632
Test Set Root Mean Squared Error: 0.4040
Test Set Mean Absolute Error: 0.2980
Test Set Mean Absolute Percentage Error: 0.0097


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


#META

In [33]:
model = LGBMRegressor(random_state=42, verbosity=-1)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_data_in_leaf': [25, 50, 100, 150, 200],
    'n_estimators' : [100, 200, 400, 600],
    'feature_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],
    'max_bin' : [50, 100, 150, 200, 400]
}

tscv = TimeSeriesSplit(n_splits=5)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv,
                           scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(Xtrain_META , ytrain_META)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(Xtrain_META)
mse = mean_squared_error(ytrain_META, y_pred)
rmse = root_mean_squared_error(ytrain_META, y_pred)
mae = mean_absolute_error(ytrain_META, y_pred)
mape = mean_absolute_percentage_error(ytrain_META, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

Best Hyperparameters: {'feature_fraction': 0.9, 'learning_rate': 0.2, 'max_bin': 150, 'max_depth': 3, 'min_data_in_leaf': 25, 'n_estimators': 600}
Test Set Mean Squared Error: 5.5447
Test Set Root Mean Squared Error: 2.3547
Test Set Mean Absolute Error: 1.7632
Test Set Mean Absolute Percentage Error: 0.0079


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
